In [ ]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import tensorflow as tf
from functools import partial

from tensorflow.keras.applications import vgg19

from tensorflow.keras.layers import Dense,Conv2D,Conv2DTranspose,BatchNormalization,Input,ReLU,Add,Lambda,LeakyReLU,Flatten
from tensorflow_addons.layers import InstanceNormalization
from tensorflow.keras.models import Model
from tensorflow import Tensor
import tensorflow as tf
import tensorflow_datasets as tfds
import keras

import cv2

In [ ]:
autotune = tf.data.experimental.AUTOTUNE
IMAGE_SIZE = [256, 256]
BATCH_SIZE=16

orig_img_size = (286, 286)
# Size of the random crops to be used during training.
input_img_size = (256, 256, 3)
# Weights initializer for the layers.
kernel_init = keras.initializers.RandomNormal(mean=0.0, stddev=0.02)
# Gamma initializer for instance normalization.
gamma_init = keras.initializers.RandomNormal(mean=0.0, stddev=0.02)

buffer_size = 256
batch_size = 4

In [ ]:

dataset, _ = tfds.load("cycle_gan/cezanne2photo", with_info=True, as_supervised=True)
train_cezanne, train_photo = dataset["trainA"], dataset["trainB"]
test_cezanne, test_photo = dataset["testA"], dataset["testB"]




def normalize_img(img):
    img = tf.cast(img, dtype=tf.float32)
    return (img / 127.5) - 1.0


def preprocess_train_image(img, label):
    img = tf.image.random_flip_left_right(img)
    img = tf.image.resize(img, [*orig_img_size])
    img = tf.image.random_crop(img, size=[*input_img_size])
    img = normalize_img(img)
    return img

def preprocess_test_image(img, label):
    img = tf.image.resize(img, [input_img_size[0], input_img_size[1]])
    img = normalize_img(img)
    return img

In [ ]:

# Apply the preprocessing operations to the training data
train_photo = (
    train_photo.map(preprocess_train_image, num_parallel_calls=autotune)
    .cache()
    .shuffle(buffer_size)
    .batch(batch_size)
)
train_cezanne = (
    train_cezanne.map(preprocess_train_image, num_parallel_calls=autotune)
    .cache()
    .shuffle(buffer_size)
    .batch(batch_size)
)

# Apply the preprocessing operations to the test data
test_photo = (
    test_photo.map(preprocess_test_image, num_parallel_calls=autotune)
    .cache()
    .shuffle(buffer_size)
    .batch(batch_size)
)
test_cezanne = (
    test_cezanne.map(preprocess_test_image, num_parallel_calls=autotune)
    .cache()
    .shuffle(buffer_size)
    .batch(batch_size)
)

In [ ]:
_, ax = plt.subplots(4, 2, figsize=(10, 15))
for i, samples in enumerate(zip(train_photo.take(4), train_photo.take(4))):
    horse = (((samples[0][0] * 127.5) + 127.5).numpy()).astype(np.uint8)
    zebra = (((samples[1][0] * 127.5) + 127.5).numpy()).astype(np.uint8)
    ax[i, 0].imshow(horse)
    ax[i, 1].imshow(horse)
plt.show()

# LOAD STYLE IMAGE

In [ ]:
style_img = plt.imread('/content/image_style.jpg')

style_img=cv2.resize(style_img,(256,256))


style_img= normalize_img(style_img)
plt.figure()
plt.imshow(style_img)
style_img=np.array(style_img)

style_image=style_img.reshape(1,256,256,3)
print(style_image.shape)

In [ ]:
vgg=vgg19.VGG19(weights='imagenet',include_top=False)
vgg.summary()

In [ ]:
content_layers=['block4_conv2']
style_layers=['block1_conv1',
            'block2_conv1',
            'block3_conv1',
            'block4_conv1',
            'block5_conv1']
content_layers_weights=[1]
style_layers_weights=[1]*5

LOSS MODEL

In [ ]:
class LossNetwork:
  def __init__(self,vgg,output_layer):
    self.initial_model = vgg
    self.output_layer = output_layer
    self.model = self.construct_model()
  
  def construct_model(self):
    output= self.initial_model.get_layer(self.output_layer).output
    input = self.initial_model.input
    model=Model(inputs=input,outputs=output)
    return model
  
  def get_model(self):
    return(self.model)

  def get_output(self,input):
    ## output to 0-255
    input=(input+1)*127.5
    outputs=self.model(vgg19.preprocess_input(input))
    return(outputs)

In [ ]:
model_style_1 = LossNetwork(vgg,'block1_conv1')
model_style_2 = LossNetwork(vgg,'block2_conv1')
model_style_3 = LossNetwork(vgg,'block3_conv1')
model_style_4 = LossNetwork(vgg,'block4_conv1')
model_style_5 = LossNetwork(vgg,'block5_conv1')

model_feature = LossNetwork(vgg,'block4_conv2')

#model=model_style_5.get_model()
#model.summary()

In [ ]:
output_style_1 = model_style_1.get_output(style_image)
output_style_2 = model_style_2.get_output(style_image)
output_style_3 = model_style_3.get_output(style_image)
output_style_4 = model_style_4.get_output(style_image)
output_style_5 = model_style_5.get_output(style_image)

output_feature = model_feature.get_output(style_image)


LOSS

Content loss

In [ ]:
def content_loss(x,y):
    return tf.reduce_mean(tf.square(x-y))

Style loss

In [ ]:
def gram_matrix(x):
    gram=tf.linalg.einsum('bijc,bijd->bcd', x, x)
    return gram/tf.cast(x.shape[1]*x.shape[2]*x.shape[3],tf.float32)

def style_loss(x,y):
    s=gram_matrix(x)
    p=gram_matrix(y)
    return tf.reduce_mean(tf.square(s-p))

In [ ]:
def preceptual_loss(real_image,style_image,predicted_image):
    
    ## Calculate output for real image 
    output_feature_real_image = model_feature.get_output(real_image)

    ## Calculate output for style image 
    output_style_style_image_1 = model_style_1.get_output(style_image)
    output_style_style_image_2 = model_style_2.get_output(style_image)
    output_style_style_image_3 = model_style_3.get_output(style_image)
    output_style_style_image_4 = model_style_4.get_output(style_image)
    output_style_style_image_5 = model_style_5.get_output(style_image)

    ## Calculate output for predicted image 
    output_style_predicted_image_1 = model_style_1.get_output(predicted_image)
    output_style_predicted_image_2 = model_style_2.get_output(predicted_image)
    output_style_predicted_image_3 = model_style_3.get_output(predicted_image)
    output_style_predicted_image_4 = model_style_4.get_output(predicted_image)
    output_style_predicted_image_5 = model_style_5.get_output(predicted_image)

    output_feature_predicted_image = model_feature.get_output(predicted_image)

    ## calculate content loss
    content_loss_1 = content_loss(output_feature_real_image,output_feature_predicted_image)
    
    ## calculate style losse
    
    style_loss_1 = style_loss(output_style_style_image_1,output_style_predicted_image_1)
    style_loss_2 = style_loss(output_style_style_image_2,output_style_predicted_image_2)
    style_loss_3 = style_loss(output_style_style_image_3,output_style_predicted_image_3)
    style_loss_4 = style_loss(output_style_style_image_4,output_style_predicted_image_4)
    style_loss_5 = style_loss(output_style_style_image_5,output_style_predicted_image_5)
    ## total loss
    
    c_loss=content_loss_1

    s_loss=style_loss_1+style_loss_2+style_loss_3+style_loss_4+style_loss_5
    #s_loss=s_loss*style_weight
    return c_loss+s_loss

## MODEL gen


In [ ]:
gamma_initializer = tf.keras.initializers.RandomNormal(mean=0.0, stddev=0.02)

def residual_block(x: Tensor) -> Tensor:
  initializer = tf.random_normal_initializer(mean=0.0, stddev=0.02, seed=None)
  x_0=x
  x = tf.pad(x_0, [[0, 0], [1, 1], [1, 1], [0, 0]], mode='REFLECT')
  y = Conv2D(kernel_size=(3,3),strides= (1,1),filters=256,padding="valid", use_bias=False,kernel_initializer=initializer)(x)
  y = BatchNormalization(gamma_initializer=gamma_initializer)(y)
  y = ReLU()(y)

  y = tf.pad(y, [[0, 0], [1, 1], [1, 1], [0, 0]], mode='REFLECT')
  y = Conv2D(kernel_size=(3,3),strides=(1,1),filters=256,padding="valid", use_bias=False,kernel_initializer=initializer)(y)
  y = BatchNormalization(gamma_initializer=gamma_initializer)(y)

  out = Add()([x_0,y])
  return out

In [ ]:
def create_generator():
  inputs = Input(shape=(256, 256, 3))
  initializer = tf.random_normal_initializer(mean=0.0, stddev=0.02, seed=None)
  ## padding 
  pad= tf.pad(inputs, [[0, 0], [3, 3], [3, 3], [0, 0]], mode='REFLECT')
  ## C7s1-64
  layer_1 = Conv2D(filters=64,kernel_size=(7,7),strides=(1,1),padding='valid',kernel_initializer=initializer,use_bias=False)(pad)
  layer_1 = BatchNormalization(gamma_initializer=gamma_initializer)(layer_1)
  layer_1 = ReLU()(layer_1)
  

  

  ## d128
  layer_2 = Conv2D(filters=128,use_bias=False,kernel_size=(3,3),strides=(2,2),padding='same',kernel_initializer=initializer)(layer_1)
  layer_2 = BatchNormalization(gamma_initializer=gamma_initializer)(layer_2)
  layer_2 = ReLU()(layer_2)
  

  ## d256
  layer_3 = Conv2D(filters=256,use_bias=False,kernel_size=(3,3),strides=(2,2),padding='same',kernel_initializer=initializer)(layer_2)
  layer_3 = BatchNormalization(gamma_initializer=gamma_initializer)(layer_3)
  layer_3 = ReLU()(layer_3)
  
  
  
  ## R256 - 1
  layer_4 = residual_block(layer_3)
  ## R256 - 2
  layer_5 = residual_block(layer_4)
  ## R256 - 3
  layer_6 = residual_block(layer_5)
  ## R256 - 4
  layer_7 = residual_block(layer_6)
   ## R256 - 5
  layer_8 = residual_block(layer_7)
  ## R256 - 6
  layer_9 = residual_block(layer_8)
  ## R256 - 7 
  layer_10 = residual_block(layer_9)
  ## R256 - 8
  layer_11 = residual_block(layer_10)
  ## R256 - 79
  layer_12 = residual_block(layer_11)

  ## u128
  layer_13 = Conv2DTranspose(filters=128,use_bias=False,kernel_size=(3,3),strides=(2,2),padding='same',kernel_initializer=initializer)(layer_12)
  layer_13 = BatchNormalization()(layer_13)
  layer_13 = ReLU()(layer_13)
  

  ## u64
  layer_14 = Conv2DTranspose(filters=64,use_bias=False,kernel_size=(3,3),strides=(2,2),padding='same',kernel_initializer=initializer)(layer_13)
  layer_14 = BatchNormalization()(layer_14)
  layer_14 = ReLU()(layer_14)
  

  ##c7s1-3
  layer_15= tf.pad(layer_14, [[0, 0], [3, 3], [3, 3], [0, 0]], mode='REFLECT')
  layer_15 = Conv2D(padding='same',filters=3,kernel_size=(7,7),strides=(1,1),activation='tanh',kernel_initializer=initializer)(layer_15)


  model = Model(inputs, layer_15)
  return(model)

In [ ]:
class FastStyleTransferModel(keras.Model):
    def __init__(
        self,
        generator,
        image_style
  
    ):
        super(FastStyleTransferModel, self).__init__()
        self.generator=generator
        self.image_style=image_style

    def compile(
        self,
        generator_optimizer,
        generator_loss_fn,
 
    ):
        super(FastStyleTransferModel, self).compile()
        self.generator_optimizer = generator_optimizer
        self.generator_loss_fn=generator_loss_fn

    def train_step(self, batch_data):
      
        with tf.GradientTape() as tape:
            predicted_image=self.generator(batch_data, training=True)
            loss=self.generator_loss_fn(batch_data,self.image_style,predicted_image)

           

        grads_generator = tape.gradient(loss, self.generator.trainable_variables)
        # Update the weights of the generators

        self.generator_optimizer.apply_gradients(
            zip(grads_generator, self.generator.trainable_variables)
        )
        

        return {
            "Perceptual loss": loss,
        }

In [ ]:
my_gen = create_generator()
my_optimizer = tf.keras.optimizers.Adam(0.001)

# Create cycle gan model
model = FastStyleTransferModel(
    generator= my_gen,
    image_style =style_image
)

# Compile the model
model.compile(
    generator_optimizer=my_optimizer,
    generator_loss_fn=preceptual_loss

)

In [ ]:
model.fit(
    x=train_photo,
    epochs=4
)

Epoch 1/4
1572/1572 [==============================] - 989s 629ms/step - Perceptual loss: 373417.7753
Epoch 2/4
1572/1572 [==============================] - 991s 630ms/step - Perceptual loss: 368007.5075
Epoch 3/4
1572/1572 [==============================] - 987s 628ms/step - Perceptual loss: 377694.9905
Epoch 4/4
1572/1572 [==============================] - 986s 627ms/step - Perceptual loss: 359116.6416


In [ ]:
model.save_weights("fast_style_transfer_weights.h5")

TEST

In [ ]:

for img in test_photo:

  
  fake_painting=my_gen(img)


 

  plt.figure(figsize=(15,15))
  plt.subplot(1,2,1)
  plt.title('Original photo')
  plt.imshow((img[0]+1)/2)
  plt.subplot(1,2,2)
  plt.title('Style applied')
  plt.imshow((fake_painting[0]+1)/2)
  
  break

  